In [ ]:
import json
import os
import sys
import getpass
import hashlib
import uuid
from datetime import datetime
from typing import Dict, Any, List

In [ ]:
DATA_FILE = "lms_data.json"

In [ ]:
def now_str():
    return datetime.now().isoformat(timespec="seconds")


def hash_password(password: str) -> str:
    return hashlib.sha256(password.encode("utf-8")).hexdigest()


def clear_screen():
    os.system("cls" if os.name == "nt" else "clear")


def pause(msg="Press Enter to continue..."):
    input(msg)